#### 01 - Setup and validate

In [1]:
from pyspark.sql import SparkSession
import os

# -----------------------------
# Env
# -----------------------------
POLARIS_URI = os.getenv("POLARIS_URI", "http://polaris:8181/api/catalog").rstrip("/")
POLARIS_OAUTH2 = os.getenv("POLARIS_OAUTH2_TOKEN_URL", "http://polaris:8181/api/catalog/v1/oauth/tokens")
POLARIS_SCOPE = os.getenv("POLARIS_SCOPE", "PRINCIPAL_ROLE:ALL")
POLARIS_CLIENT_ID = os.getenv("POLARIS_CLIENT_ID", "admin")
POLARIS_CLIENT_SECRET = os.getenv("POLARIS_CLIENT_SECRET", "password")

SPARK_MASTER = os.getenv("SPARK_MASTER", "spark://spark-master:7077")
DRIVER_HOST = os.getenv("SPARK_DRIVER_HOST", "jupyter")

# pin vaste driver ports (voorkomt flaky executor callbacks)
DRIVER_PORT = os.getenv("SPARK_DRIVER_PORT", "7078")
BLOCKMANAGER_PORT = os.getenv("SPARK_BLOCKMANAGER_PORT", "7079")
SPARK_UI_PORT = os.getenv("SPARK_UI_PORT", "4040")

S3_ENDPOINT = os.getenv("S3_ENDPOINT", "http://minio:9000")
S3_ACCESS_KEY = os.getenv("MINIO_ROOT_USER", "minioadmin")
S3_SECRET_KEY = os.getenv("MINIO_ROOT_PASSWORD", "minioadmin")

POLARIS_WAREHOUSE = os.getenv("POLARIS_WAREHOUSE", "polaris").strip()
AWS_REGION = os.getenv("AWS_REGION", "us-east-1").strip()

# -----------------------------
# Stop oude sessie
# -----------------------------
if "spark" in locals():
    try:
        spark.stop()
    except Exception:
        pass

print(f"🔗 SPARK_MASTER        : {SPARK_MASTER}")
print(f"🧷 DRIVER_HOST         : {DRIVER_HOST}")
print(f"🔌 DRIVER_PORT         : {DRIVER_PORT}")
print(f"📦 BLOCKMANAGER_PORT   : {BLOCKMANAGER_PORT}")
print(f"🖥️  SPARK_UI_PORT       : {SPARK_UI_PORT}")
print(f"🧭 POLARIS_URI         : {POLARIS_URI}")
print(f"🏷️  POLARIS_WAREHOUSE   : {POLARIS_WAREHOUSE}")
print(f"🪣 S3 endpoint         : {S3_ENDPOINT}")
print(f"🌍 AWS_REGION          : {AWS_REGION}")

# -----------------------------
# SparkSession
# -----------------------------
builder = (
    SparkSession.builder
    .appName("Lakehouse-Unplugged")
    .master(SPARK_MASTER)

    # netwerk/driver (deterministisch)
    .config("spark.driver.host", DRIVER_HOST)
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.port", str(DRIVER_PORT))
    .config("spark.blockManager.port", str(BLOCKMANAGER_PORT))
    .config("spark.ui.port", str(SPARK_UI_PORT))
    .config("spark.port.maxRetries", "32")
    # optioneel: minder 'hangende' progress regels in notebooks
    .config("spark.ui.showConsoleProgress", "false")

    # core spark
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

    # Polaris default catalog
    .config("spark.sql.defaultCatalog", "polaris")
    .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.polaris.type", "rest")
    .config("spark.sql.catalog.polaris.uri", POLARIS_URI)
    .config("spark.sql.catalog.polaris.warehouse", POLARIS_WAREHOUSE)

    # OAuth
    .config("spark.sql.catalog.polaris.rest.auth.type", "oauth2")
    .config("spark.sql.catalog.polaris.credential", f"{POLARIS_CLIENT_ID}:{POLARIS_CLIENT_SECRET}")
    .config("spark.sql.catalog.polaris.oauth2-server-uri", POLARIS_OAUTH2)
    .config("spark.sql.catalog.polaris.scope", POLARIS_SCOPE)
    .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")

    # Iceberg S3FileIO (AWS SDK v2)
    .config("spark.sql.catalog.polaris.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.polaris.s3.endpoint", S3_ENDPOINT)
    .config("spark.sql.catalog.polaris.s3.path-style-access", "true")
    .config("spark.sql.catalog.polaris.s3.access-key-id", S3_ACCESS_KEY)
    .config("spark.sql.catalog.polaris.s3.secret-access-key", S3_SECRET_KEY)
    .config("spark.sql.catalog.polaris.s3.region", AWS_REGION)

    # region doorgeven aan executors/driver
    .config("spark.executorEnv.AWS_REGION", AWS_REGION)
    .config("spark.executorEnv.AWS_DEFAULT_REGION", AWS_REGION)
    .config("spark.driverEnv.AWS_REGION", AWS_REGION)
    .config("spark.driverEnv.AWS_DEFAULT_REGION", AWS_REGION)

    # S3A (Hadoop, AWS SDK v1) voor landing reads
    .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.endpoint.region", AWS_REGION)

    # --- laptop limits (pas aan op jouw worker resources) ---
    .config("spark.cores.max", "2")
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "1g")
    .config("spark.executor.memoryOverhead", "256m")
    .config("spark.driver.memory", "1g")
    .config("spark.driver.memoryOverhead", "256m")

    .config("spark.sql.shuffle.partitions", "8")
    .config("spark.sql.adaptive.enabled", "true")
)

pkgs = (os.getenv("SPARK_JARS_PACKAGES") or "").strip()
if pkgs:
    print(f"⚠️ spark.jars.packages staat aan (driver downloadt deps): {pkgs}")
    builder = builder.config("spark.jars.packages", pkgs)

spark = builder.getOrCreate()

print("✅ Spark up.")
print("defaultCatalog =", spark.conf.get("spark.sql.defaultCatalog"))
print("polaris.warehouse =", spark.conf.get("spark.sql.catalog.polaris.warehouse"))
print("spark.driver.host =", spark.conf.get("spark.driver.host", "<unset>"))
print("spark.driver.port =", spark.conf.get("spark.driver.port", "<unset>"))
print("AWS_REGION (driver env) =", os.getenv("AWS_REGION"))
print("🧪 Sanity spark.range(10).count() =", spark.range(10).count())


🔗 SPARK_MASTER        : spark://spark-master:7077
🧷 DRIVER_HOST         : jupyter
🔌 DRIVER_PORT         : 7078
📦 BLOCKMANAGER_PORT   : 7079
🖥️  SPARK_UI_PORT       : 4040
🧭 POLARIS_URI         : http://polaris:8181/api/catalog
🏷️  POLARIS_WAREHOUSE   : polaris
🪣 S3 endpoint         : http://minio:9000
🌍 AWS_REGION          : dummy


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/09 09:15:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark up.
defaultCatalog = polaris
polaris.warehouse = polaris
spark.driver.host = jupyter
spark.driver.port = 7078
AWS_REGION (driver env) = dummy
🧪 Sanity spark.range(10).count() = 10


#### 02 - Parkeer bestanden in de landingzone

In [2]:
import os
import boto3
from pathlib import Path
from botocore.exceptions import ClientError

# ======================================================================
# 0️⃣ Helper: zoek automatisch lokaal data-bestand
# ======================================================================
def find_data_file(filename: str) -> Path:
    p = Path.cwd()
    for _ in range(6):
        candidate = p / "data" / filename
        if candidate.exists():
            return candidate
        p = p.parent
    raise FileNotFoundError(f"❌ Kon '{filename}' niet vinden in een 'data' map vanaf {Path.cwd()}.")

# ======================================================================
# 1️⃣ Config (uit env waar kan)
# ======================================================================
local_file = find_data_file("gekentekendevoertuigen_sample.json")

bucket = os.getenv("MINIO_BUCKET", "warehouse")
prefix = os.getenv("MINIO_PREFIX", "landing")

endpoint = os.getenv("S3_ENDPOINT", "http://minio:9000")
access_key = os.getenv("MINIO_ROOT_USER", "minioadmin")
secret_key = os.getenv("MINIO_ROOT_PASSWORD", "minioadmin")
region = os.getenv("AWS_REGION", "us-east-1")

object_key = f"{prefix}/{local_file.name}"
s3a_uri = f"s3a://{bucket}/{object_key}"

print(f"📄 Lokaal bestand : {local_file}")
print(f"⬆️ Upload naar    : s3://{bucket}/{object_key}")
print(f"📥 Spark read via : {s3a_uri}")
print(f"🪣 MinIO endpoint : {endpoint}")

# ======================================================================
# 2️⃣ MinIO client via boto3 (S3 API)
# ======================================================================
s3 = boto3.client(
    "s3",
    endpoint_url=endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name=region,
)

# Bucket check (minio-setup maakt 'warehouse' al aan, maar dit maakt het robuust)
try:
    s3.head_bucket(Bucket=bucket)
except ClientError:
    print(f"ℹ️ Bucket '{bucket}' bestaat nog niet, maak 'm aan...")
    s3.create_bucket(Bucket=bucket)

# Upload bestand
s3.upload_file(str(local_file), bucket, object_key)
print("✅ Upload gelukt.")

# ======================================================================
# 3️⃣ Verify: lijst objecten in prefix
# ======================================================================
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
print("📦 Objecten in MinIO:")
for item in response.get("Contents", []):
    print(" -", item["Key"])

# ======================================================================
# 4️⃣ Spark read via S3A
# ======================================================================
df = spark.read.option("multiline", "true").json(s3a_uri)

print(f"📊 Aantal records: {df.count():,}")
df.printSchema()
df.show(5, truncate=False)


📄 Lokaal bestand : /workspace/data/gekentekendevoertuigen_sample.json
⬆️ Upload naar    : s3://warehouse/landing/gekentekendevoertuigen_sample.json
📥 Spark read via : s3a://warehouse/landing/gekentekendevoertuigen_sample.json
🪣 MinIO endpoint : http://minio:9000
ℹ️ Bucket 'warehouse' bestaat nog niet, maak 'm aan...
✅ Upload gelukt.
📦 Objecten in MinIO:
 - landing/gekentekendevoertuigen_sample.json


26/01/09 09:16:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


📊 Aantal records: 10,000
root
 |-- aanhangwagen_autonoom_geremd: string (nullable = true)
 |-- aanhangwagen_middenas_geremd: string (nullable = true)
 |-- aantal_cilinders: string (nullable = true)
 |-- aantal_deuren: string (nullable = true)
 |-- aantal_rolstoelplaatsen: string (nullable = true)
 |-- aantal_staanplaatsen: string (nullable = true)
 |-- aantal_wielen: string (nullable = true)
 |-- aantal_zitplaatsen: string (nullable = true)
 |-- afstand_hart_koppeling_tot_achterzijde_voertuig: string (nullable = true)
 |-- afstand_voorzijde_voertuig_tot_hart_koppeling: string (nullable = true)
 |-- afwijkende_maximum_snelheid: string (nullable = true)
 |-- api_gekentekende_voertuigen_assen: string (nullable = true)
 |-- api_gekentekende_voertuigen_brandstof: string (nullable = true)
 |-- api_gekentekende_voertuigen_carrosserie: string (nullable = true)
 |-- api_gekentekende_voertuigen_carrosserie_specifiek: string (nullable = true)
 |-- api_gekentekende_voertuigen_voertuigklasse: strin

26/01/09 09:16:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------------------+----------------------------+----------------+-------------+-----------------------+--------------------+-------------+------------------+-----------------------------------------------+---------------------------------------------+---------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-------------------------------------------------+-----------------------------------------------+-------+------------------------+------------------------+---------+--------------+--------------+-----------------------------------+----------------------------------------+-------------------------------------------+----------------------+-------------------------+--------------------+-----------------------+------------+---------------------------------------+--------------------------+-------------------------------------+----------------+---------------+----

#### 03 - Ingest into bronze table

In [3]:
# ======================================================================
# 🧊 Write Bronze Iceberg table explicitly to Polaris (Spark-safe)
# ======================================================================

catalog = "polaris"
namespace = "bronze"
table_name = "gekentekendevoertuigen"

ns_fqn = f"{catalog}.{namespace}"
table_fqn = f"{catalog}.{namespace}.{table_name}"

# 0) Sanity checks
print("🔍 Spark context")
spark.sql("SELECT current_catalog() AS catalog, current_database() AS namespace").show(truncate=False)
spark.sql("SHOW CATALOGS").show(truncate=False)

# 1) Create namespace explicitly in Polaris
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {ns_fqn}")

# 2) Write Iceberg table
print(f"🧊 Writing Iceberg table: {table_fqn}")

(
    df.writeTo(table_fqn)
      .using("iceberg")
      .option("format-version", "2")
      .createOrReplace()
)

# 3) Refresh + verify
spark.catalog.refreshTable(table_fqn)

print(f"📋 Tables in {ns_fqn}:")
spark.sql(f"SHOW TABLES IN {ns_fqn}").show(truncate=False)

print("🔁 Sample from Bronze:")
spark.read.table(table_fqn).show(5, truncate=False)


🔍 Spark context
+-------+---------+
|catalog|namespace|
+-------+---------+
|polaris|         |
+-------+---------+

+-------------+
|catalog      |
+-------------+
|polaris      |
|spark_catalog|
+-------------+

🧊 Writing Iceberg table: polaris.bronze.gekentekendevoertuigen
📋 Tables in polaris.bronze:
+---------+----------------------+-----------+
|namespace|tableName             |isTemporary|
+---------+----------------------+-----------+
|bronze   |gekentekendevoertuigen|false      |
+---------+----------------------+-----------+

🔁 Sample from Bronze:
+----------------------------+----------------------------+----------------+-------------+-----------------------+--------------------+-------------+------------------+-----------------------------------------------+---------------------------------------------+---------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+---

#### 04 - Check Silver via Polaris catalog

In [6]:
CATALOG = "polaris"
SCHEMA = "dbt_demo_silver"

tables = [
    "md_voertuig",
    "rd_motorconfiguratie",
    "rd_rdw_referenties",
    "rd_voertuig_classificaties",
    "rd_voertuig_statuswaarden",
]

print("📦 Tabellen in silver:\n")
spark.sql(f"SHOW TABLES IN {CATALOG}.{SCHEMA}").show(truncate=False)

for tbl in tables:
    fqtn = f"{CATALOG}.{SCHEMA}.{tbl}"
    print(f"\n🔍 {fqtn}")

    # schema
    spark.sql(f"DESCRIBE TABLE {fqtn}").show(truncate=False)

    # row count
    spark.sql(f"SELECT COUNT(*) AS rows FROM {fqtn}").show()

    # sample
    spark.sql(f"SELECT * FROM {fqtn} LIMIT 10").show(truncate=False)


📦 Tabellen in silver:

+---------------+--------------------------+-----------+
|namespace      |tableName                 |isTemporary|
+---------------+--------------------------+-----------+
|dbt_demo_silver|rd_rdw_referenties        |false      |
|dbt_demo_silver|rd_voertuig_classificaties|false      |
|dbt_demo_silver|md_voertuig               |false      |
|dbt_demo_silver|rd_voertuig_statuswaarden |false      |
|dbt_demo_silver|rd_motorconfiguratie      |false      |
+---------------+--------------------------+-----------+


🔍 polaris.dbt_demo_silver.md_voertuig
+----------------------------------------+---------+-------+
|col_name                                |data_type|comment|
+----------------------------------------+---------+-------+
|kenteken                                |string   |NULL   |
|voertuigsoort                           |string   |NULL   |
|merk                                    |string   |NULL   |
|handelsbenaming                         |string   |NULL  

+--------+--------------------+-------------+-------------------------------+------------------+---------+------------------------+------+-------+------+---------+--------------------+--------------+----------------------------+----------------------------+---------------------------------+-----------------------------+----------------------------+-----------------------+----------------------------------------+-----------------------+-----------------------+----------------------+--------------------------------+
|kenteken|voertuigsoort       |merk         |handelsbenaming                |uitvoering        |variant  |inrichting              |lengte|breedte|hoogte|wielbasis|massa_ledig_voertuig|massa_rijklaar|massa_bedrijfsklaar_maximaal|massa_bedrijfsklaar_minimaal|toegestane_maximum_massa_voertuig|technische_max_massa_voertuig|maximale_constructiesnelheid|datum_eerste_toelating |datum_eerste_tenaamstelling_in_nederland|datum_tenaamstelling   |vervaldatum_apk        |vervaldatum_tacho

+--------------------------------+----------------+--------------+-------------------+------------------------+----------------------+---------------------------+
|motorconfiguratie_id            |aantal_cilinders|cilinderinhoud|type_gasinstallatie|zuinigheidsclassificatie|vermogen_massarijklaar|afwijkende_maximum_snelheid|
+--------------------------------+----------------+--------------+-------------------+------------------------+----------------------+---------------------------+
|59d7d64dbcc254544f04cdb81ccff699|                |              |                   |                        |                      |NULL                       |
|500f694e65de11820712b8edf3422789|                |              |                   |                        |0.00                  |NULL                       |
|168b6272a97a4896127841d86bb0730f|                |              |                   |                        |0.01                  |NULL                       |
|7d6441b796e2ce1adac08

#### 05 - Check Gold via Polaris catalog

In [7]:
CATALOG = "polaris"
SCHEMA = "dbt_demo_gold"

tables = [
    "dim_classificatie",
    "dim_rdw_referentie",
    "dim_status",
    "fact_registratie_events",
    "fact_keuring_events",
]

print("📦 Tabellen in gold:\n")
spark.sql(f"SHOW TABLES IN {CATALOG}.{SCHEMA}").show(truncate=False)

for tbl in tables:
    fqtn = f"{CATALOG}.{SCHEMA}.{tbl}"
    print(f"\n🔍 {fqtn}")

    # schema
    spark.sql(f"DESCRIBE TABLE {fqtn}").show(truncate=False)

    # row count
    spark.sql(f"SELECT COUNT(*) AS rows FROM {fqtn}").show()

    # sample
    spark.sql(f"SELECT * FROM {fqtn} LIMIT 10").show(truncate=False)


📦 Tabellen in gold:

+-------------+-----------------------+-----------+
|namespace    |tableName              |isTemporary|
+-------------+-----------------------+-----------+
|dbt_demo_gold|dim_rdw_referentie     |false      |
|dbt_demo_gold|dim_motorconfiguratie  |false      |
|dbt_demo_gold|dim_classificatie      |false      |
|dbt_demo_gold|dim_status             |false      |
|dbt_demo_gold|fact_keuring_events    |false      |
|dbt_demo_gold|fact_registratie_events|false      |
+-------------+-----------------------+-----------+


🔍 polaris.dbt_demo_gold.dim_classificatie
+---------------------------------------+---------+-------+
|col_name                               |data_type|comment|
+---------------------------------------+---------+-------+
|kenteken                               |string   |NULL   |
|europese_voertuigcategorie             |string   |NULL   |
|europese_voertuigcategorie_toevoeging  |string   |NULL   |
|subcategorie_nederland                 |string   |NULL